In [169]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. Intro
1. Use InferNet to generate params values required by GT3.
# 1. Preparations
## 1.1 Global

In [170]:
import torch

import numpy as np
import random
import pandas as pd

from MLP.models import MLP_GT3_2
from importlib import reload

from MLP.my_collate_fn import my_collate_fn_InferNet
from torch.utils.data import DataLoader, SequentialSampler
from MLP.mydataset_GT2 import myDataset

from MLP.Config import config_GT_gen
reload(config_GT_gen)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

bound_alpha = torch.tensor([-0.3,0.3],device=device)
bound_labda = torch.tensor([0.01,18],device=device)

opt = config_GT_gen.DefaultConfig()

## 1.2 相关path

In [171]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(opt.seed)

dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.params_opitim_path,opt.data_key_path, opt.NLL_metric_path)

data_idx = np.arange(0,dataset.__len__()).tolist()
print("seed=",opt.seed)
print("arti=",opt.ARTIFICIAL)
print("GT_w_Params=",opt.GT_w_Params)

seed= 407
arti= False
GT_w_Params= 3


# 2. dataloader

In [172]:
data_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False, sampler=SequentialSampler(data_idx), collate_fn=my_collate_fn_InferNet)

# 3. Load and Generate

In [1]:
mlp = MLP_GT3_2()
model_data = torch.load(opt.model_params_MLP)
mlp.load_state_dict(model_data)
mlp = mlp.to(device=device)

NameError: name 'MLP_GT3_2' is not defined

In [174]:
if opt.GT_w_Params == 2:
    params_arr = np.array([0.]*(dataset.__len__()*2)).reshape(-1,2)
    params = pd.DataFrame(params_arr,columns = ['alpha','labda'])
elif opt.GT_w_Params == 3:
    params_arr = np.array([0.]*(dataset.__len__()*1)).reshape(-1,1)
    params = pd.DataFrame(params_arr,columns = ['alpha'])

In [175]:
model=mlp.eval()
if opt.GT_w_Params == 2:
    for batch_id, data in enumerate(data_loader):

        input_data, _,_, _, _, _= data
        # Do the inference
        input_data = input_data.to(device)

        Alpha, Labda = mlp(input_data)

        Alpha = Alpha.detach().cpu().numpy()
        Labda = Labda.detach().cpu().numpy()

        params.loc[batch_id,'alpha'] = Alpha
        params.loc[batch_id,'labda'] = Labda

if opt.GT_w_Params == 3:
    for batch_id, data in enumerate(data_loader):

        input_data, _,_, _, _, _= data
        # Do the inference
        input_data = input_data.to(device)

        Alpha= mlp(input_data)

        Alpha = Alpha.detach().cpu().numpy()

        params.loc[batch_id,'alpha'] = Alpha

print("Done")
print(params.head())

Done
      alpha
0  0.001608
1  0.000522
2  0.000522
3  0.000522
4  0.000522


- save

In [176]:
print(opt.params_gen_path)

../../data/SA_PT/params_GT3_seed=407.csv


In [177]:
params.to_csv(opt.params_gen_path,encoding="utf-8", header=True,index=False)